In [1]:
import pandas as pd
from pathlib import Path
from geopy.distance import geodesic
import numpy as np

In [ ]:
#Connect to the database
conn_string = "host='localhost' dbname='glacier_data' user='postgres' password='REDACTED'"
#password has been removed for security reasons
conn = psycopg2.connect(conn_string)
print("Database opened successfully")
cursor = conn.cursor();
query = 'select * from yourdatabase'
cursor.execute(query)
colnames = [desc[0] for desc in cursor.description]
df = pd.DataFrame(cursor.fetchall(), columns=colnames)
cursor.close()
print('Database closed successfully!')

In [2]:
#read in csv
glaciers_raw = pd.read_csv(Path("../Data/AK_1985_to_2000_overall_glacier_covered_area.csv"))
glaciers_raw.head()

,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,...,Lmax,Status,Connect,Form,TermType,Surging,Linkages,Name,GN_area,GN_comp_ye
0,RGI60-01.00002,G213332E63404N,20090703,-9999999,-146.668,63.404,1,2,0.558,1713,...,1197,0,0,0,0,9,9,NaN,0.5922,2000
1,RGI60-01.00003,G213920E63376N,20090703,-9999999,-146.080,63.376,1,2,1.685,1609,...,2106,0,0,0,0,9,9,NaN,1.8045,2000
2,RGI60-01.00004,G213880E63381N,20090703,-9999999,-146.120,63.381,1,2,3.681,1273,...,4175,0,0,0,0,9,9,NaN,3.6765,2000
3,RGI60-01.00005,G212943E63551N,20090703,-9999999,-147.057,63.551,1,2,2.573,1494,...,2981,0,0,0,0,9,9,NaN,2.6064,2000
4,RGI60-01.00006,G213756E63571N,20090703,-9999999,-146.244,63.571,1,2,10.470,1201,...,10518,0,0,0,0,9,9,NaN,7.7220,2000


In [3]:
#glaciers only
glaciers=glaciers_raw[glaciers_raw['Form'] == 0]
#clarifying column names and converting to Imperial 
glaciers['Id'] = glaciers_raw['GLIMSId']
glaciers['Height (m)'] = glaciers_raw['Zmax'] - glaciers_raw['Zmin']
glaciers['Height (ft)'] = glaciers['Height (m)'] * 3.281
glaciers['Length (ft)'] = glaciers_raw['Lmax'] * 3.281
glaciers['Area (mi^2)'] = glaciers_raw['GN_area'] / 2.59
glaciers['Year'] = glaciers_raw['GN_comp_ye']
glaciers['Lat'] = glaciers_raw['CenLat']
glaciers['Lon'] = glaciers_raw['CenLon']
#removing excess columns and sorting by Id
glaciers=glaciers.sort_values(by=["Id", "Year"]).reset_index()
glaciers=glaciers[['Id','Name','Year', 'Lat', 'Lon', 'Area (mi^2)', 'Height (ft)', 'Length (ft)']]
glaciers.head()

,Id,Name,Year,Lat,Lon,Area (mi^2),Height (ft),Length (ft)
0,G193000E53881N,NaN,1986,53.881,-167.0,0.031969,2723.23,6949.158
1,G193000E53881N,NaN,1988,53.881,-167.0,0.031969,2723.23,6949.158
2,G193000E53881N,NaN,1990,53.881,-167.0,0.031969,2723.23,6949.158
3,G193000E53881N,NaN,1992,53.881,-167.0,0.031969,2723.23,6949.158
4,G193000E53881N,NaN,1994,53.881,-167.0,0.031969,2723.23,6949.158


In [4]:
#doing just difference between 1985-2020
twodecade=glaciers[glaciers['Year'].isin([1986, 2020])]
td_1986 = twodecade[twodecade['Year'] == 1986]
td_2020 = twodecade[twodecade['Year'] == 2020]
twodecade = pd.concat([td_1986, td_2020])
twodecade=twodecade.sort_values(by=["Id", "Year"]).reset_index(drop=True)
twodecade.head()

,Id,Name,Year,Lat,Lon,Area (mi^2),Height (ft),Length (ft)
0,G193000E53881N,NaN,1986,53.881000,-167.000000,0.031969,2723.230,6949.158
1,G193000E53881N,NaN,2020,53.881000,-167.000000,0.103552,2723.230,6949.158
2,G193008E53893N,NaN,1986,53.893000,-166.992000,0.021544,3238.347,5610.510
3,G193008E53893N,NaN,2020,53.893000,-166.992000,0.085830,3238.347,5610.510
4,G193023E53891N,NaN,1986,53.891464,-166.976868,0.420811,3303.967,11040.565


In [5]:
#calculating changes between 1985 & 2020
td_1986 = twodecade[twodecade['Year'] == 1986]
td_2020 = twodecade[twodecade['Year'] == 2020]
glaciers = pd.merge(td_1986, td_2020, on=['Id','Name'], suffixes=('_1986', '_2020'))
glaciers['Area Shrinkage (mi^2)'] = glaciers['Area (mi^2)_1986']-glaciers['Area (mi^2)_2020']
#since lat, lon, height, and length didnt change as shown earlier, differences not needed
glaciers['Lat']=glaciers['Lat_1986']
glaciers['Lon']=glaciers['Lon_1986']
glaciers['Height (ft)']=glaciers['Height (ft)_1986']
glaciers['Length (ft)']=glaciers['Length (ft)_1986']
glaciers=glaciers[['Id','Name','Lat','Lon','Area (mi^2)_1986','Area (mi^2)_2020','Area Shrinkage (mi^2)','Height (ft)','Length (ft)']]
glaciers.head()

,Id,Name,Lat,Lon,Area (mi^2)_1986,Area (mi^2)_2020,Area Shrinkage (mi^2),Height (ft),Length (ft)
0,G193000E53881N,NaN,53.881000,-167.000000,0.031969,0.103552,-0.071583,2723.230,6949.158
1,G193008E53893N,NaN,53.893000,-166.992000,0.021544,0.085830,-0.064286,3238.347,5610.510
2,G193023E53891N,NaN,53.891464,-166.976868,0.420811,0.461120,-0.040309,3303.967,11040.565
3,G193025E53876N,NaN,53.876000,-166.975000,2.621120,2.530772,0.090347,4481.846,19833.645
4,G193041E53868N,NaN,53.868000,-166.959000,2.418533,2.636757,-0.218224,4170.151,17356.490


In [6]:
#read in combined weather csv created in "Cleaning For Unsupervised AI"
annual_weather = pd.read_csv(Path("../Data/weathercombined.csv"))
annual_weather.head()

,Year,Lat,Lon,Monthly Average Mean Temperature (degF),Monthly Total Precipitation (in),Monthly Total Snowfall (in)
0,2000,60.383846,-144.045262,37.771548,4.956585,9.745098
1,2001,60.383846,-144.045262,37.466310,5.071205,10.638095
2,2002,60.383846,-144.045262,38.556786,5.218313,11.631481
3,2003,60.383846,-144.045262,38.369524,4.810000,10.921154
4,2004,60.383846,-144.045262,38.616548,4.888333,12.683929


In [7]:
# Function to calculate the closest city using geodesic
def find_closest_city(Lat_glacier, Lon_glacier, cities_df, distance_threshold=3):
    closest_distance = float('inf')  # Start with an infinitely large distance
    closest_city = None
    for _, row in cities_df.iterrows():
        # Calculate the difference in Lat/Lon
        Lat_diff = abs(Lat_glacier - row['Lat'])
        Lon_diff = abs(Lon_glacier - row['Lon'])
        # Check if the Lat/Lon differences are within the 3-degree threshold
        if Lat_diff <= distance_threshold and Lon_diff <= distance_threshold:
            # Compute the geodesic distance between the glacier and the city
            distance = geodesic((Lat_glacier, Lon_glacier), (row['Lat'], row['Lon'])).km
            # If the distance is the smallest so far, update the closest city
            if distance < closest_distance:
                closest_distance = distance
                closest_city = row
    return closest_city

In [8]:
#note: this code will take about 10 mins to run
closest_lats = []
closest_lons = []
# For each glacier, find the closest city
def find_closest_city(Lat_glacier, Lon_glacier, cities_df, distance_threshold=3):
    # Calculate the differences in latitudes and longitudes
    lat_diff = abs(cities_df['Lat'] - Lat_glacier)
    lon_diff = abs(cities_df['Lon'] - Lon_glacier)
    # Filter cities that are within the 3-degree threshold in both lat and lon
    cities_within_threshold = cities_df[(lat_diff <= distance_threshold) & (lon_diff <= distance_threshold)]
    if cities_within_threshold.empty:
        return np.nan, np.nan
    # Compute the geodesic distance for each filtered city
    cities_within_threshold['distance'] = cities_within_threshold.apply(
        lambda row: geodesic((Lat_glacier, Lon_glacier), (row['Lat'], row['Lon'])).km, axis=1)
    # Find the city with the minimum distance
    closest_city = cities_within_threshold.loc[cities_within_threshold['distance'].idxmin()]
    return closest_city['Lat'], closest_city['Lon']
# Apply the optimized function to the 'glaciers' DataFrame to find the closest city for each glacier
glaciers['closest_lat'], glaciers['closest_lon'] = zip(*glaciers.apply(
    lambda row: find_closest_city(row['Lat'], row['Lon'], annual_weather), axis=1))
glaciers.head(10)

,Id,Name,Lat,Lon,Area (mi^2)_1986,Area (mi^2)_2020,Area Shrinkage (mi^2),Height (ft),Length (ft),closest_lat,closest_lon
0,G193000E53881N,NaN,53.881000,-167.000000,0.031969,0.103552,-0.071583,2723.230,6949.158,NaN,NaN
1,G193008E53893N,NaN,53.893000,-166.992000,0.021544,0.085830,-0.064286,3238.347,5610.510,NaN,NaN
2,G193023E53891N,NaN,53.891464,-166.976868,0.420811,0.461120,-0.040309,3303.967,11040.565,NaN,NaN
3,G193025E53876N,NaN,53.876000,-166.975000,2.621120,2.530772,0.090347,4481.846,19833.645,NaN,NaN
4,G193041E53868N,NaN,53.868000,-166.959000,2.418533,2.636757,-0.218224,4170.151,17356.490,NaN,NaN
5,G193042E53902N,NaN,53.902000,-166.958000,0.533745,0.572317,-0.038571,2949.619,8586.377,NaN,NaN
6,G193053E53910N,NaN,53.910000,-166.947000,0.186602,0.131351,0.055251,2221.237,5613.791,NaN,NaN
7,G193060E53903N,NaN,53.903000,-166.940000,0.407606,0.521236,-0.113629,2890.561,10197.348,NaN,NaN
8,G193071E53864N,NaN,53.864000,-166.929000,1.081390,1.355212,-0.273822,3041.487,10007.050,NaN,NaN
9,G193073E53899N,NaN,53.899000,-166.927000,1.311081,1.580734,-0.269653,3090.702,13104.314,NaN,NaN


In [9]:
#cleaning the new data
close_glaciers=glaciers.dropna(subset=['closest_lat'])
close_glaciers=close_glaciers.dropna(subset=['closest_lon'])
close_glaciers['GLat'] = close_glaciers['Lat']
close_glaciers['GLon'] = close_glaciers['Lon']
close_glaciers['Lat'] =close_glaciers['closest_lat']
close_glaciers['Lon'] =close_glaciers['closest_lon']
close_glaciers=close_glaciers[['Lat', 'Lon', 'Area (mi^2)_1986', 'Area (mi^2)_2020', 'Area Shrinkage (mi^2)', 'Height (ft)', 'Length (ft)']].reset_index(drop=True)
close_glaciers.head(10)

,Lat,Lon,Area (mi^2)_1986,Area (mi^2)_2020,Area Shrinkage (mi^2),Height (ft),Length (ft)
0,60.383846,-144.045262,0.120579,0.041351,0.079228,1086.011,1594.566
1,60.383846,-144.045262,0.167490,0.108069,0.059421,1099.135,1991.567
2,60.383846,-144.045262,0.439575,0.205367,0.234208,1328.805,3412.240
3,60.383846,-144.045262,0.236293,0.205019,0.031274,1207.408,4888.690
4,60.383846,-144.045262,0.275212,0.112587,0.162625,882.589,2719.949
5,60.383846,-144.045262,0.620965,0.290502,0.330463,1466.607,3901.109
6,60.383846,-144.045262,0.110849,0.080618,0.030232,711.977,1571.599
7,60.383846,-144.045262,0.174440,0.040309,0.134131,1637.219,2526.370
8,60.383846,-144.045262,0.283900,0.135869,0.148031,1653.624,6466.851
9,60.383846,-144.045262,0.322471,0.113977,0.208494,1765.178,5124.922


In [10]:
#shape of this dataset
close_glaciers.shape

(3554, 7)

In [11]:
#down to about 3500 rows of data; still plenty
merged=pd.merge(annual_weather, close_glaciers, on=['Lat','Lon'], how='outer')
merged=merged[['Lat','Lon','Monthly Average Mean Temperature (degF)', 'Monthly Total Precipitation (in)', 'Monthly Total Snowfall (in)', 'Area (mi^2)_1986', 'Area (mi^2)_2020', 'Area Shrinkage (mi^2)', 'Height (ft)', 'Length (ft)']]
merged.head()

,Lat,Lon,Monthly Average Mean Temperature (degF),Monthly Total Precipitation (in),Monthly Total Snowfall (in),Area (mi^2)_1986,Area (mi^2)_2020,Area Shrinkage (mi^2),Height (ft),Length (ft)
0,60.383846,-144.045262,37.771548,4.956585,9.745098,0.120579,0.041351,0.079228,1086.011,1594.566
1,60.383846,-144.045262,37.771548,4.956585,9.745098,0.167490,0.108069,0.059421,1099.135,1991.567
2,60.383846,-144.045262,37.771548,4.956585,9.745098,0.439575,0.205367,0.234208,1328.805,3412.240
3,60.383846,-144.045262,37.771548,4.956585,9.745098,0.236293,0.205019,0.031274,1207.408,4888.690
4,60.383846,-144.045262,37.771548,4.956585,9.745098,0.275212,0.112587,0.162625,882.589,2719.949


In [12]:
#exporting for UML use
merged.to_csv("../Data/SecUMLprep.csv", index=False, header=True)

In [ ]:
#..